In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
import json
from tqdm import tqdm
import random

import os
import sys
import copy
sys.path.append('..')

from relations import estimate
from util import model_utils
import baukit
import transformers

## Load the model on `GPU:0`

In [3]:
######################################################################################################################
MODEL_NAME = "EleutherAI/gpt-neox-20b" # options gpt2-{} | "EleutherAI/gpt-neox-20b" | "EleutherAI/gpt-j-6B"

layer_name_format = "gpt_neox.layers.{}"
final_layer_norm = "gpt_neox.final_layer_norm"
unembed = "embed_out"
num_layer_field = "num_hidden_layers"

# layer_name_format = "transformer.h.{}"
# final_layer_norm = "transformer.ln_f"
# unembed = "lm_head"
# num_layer_field = "n_layer"
######################################################################################################################



mt = model_utils.ModelAndTokenizer(MODEL_NAME, low_cpu_mem_usage=True, torch_dtype=torch.float16)

model = mt.model
tokenizer = mt.tokenizer
tokenizer.pad_token = tokenizer.eos_token

print(f"{MODEL_NAME} ==> device: {model.device}, memory: {model.get_memory_footprint()}")

EleutherAI/gpt-neox-20b ==> device: cuda:0, memory: 41293685880


## Load the later part of the model on `GPU:1`

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# path_name = "gpt2-medium__last_9_layers"
# path_name = "gpt2-xl__last_25_layers"
path_name = "EleutherAI/gpt-neox-20b__last_22_layers"

part_model = AutoModelForCausalLM.from_pretrained(
                path_name, low_cpu_mem_usage=True, torch_dtype=torch.float16
            )
part_model = part_model.eval().cuda('cuda:1')

part_layer_names = [layer_name_format.format(idx) for idx in range(getattr(part_model.config, num_layer_field))]
print(f"last {getattr(part_model.config, num_layer_field)} layers ==> device: {part_model.device}, memory: {part_model.get_memory_footprint()}")

last 22 layers ==> device: cuda:1, memory: 21266563644


In [5]:
#################################################################
break_layer_idx = getattr(model.config, num_layer_field) - getattr(part_model.config, num_layer_field)
#################################################################
break_layer_idx

22

### The model is loaded in `eval` mode. Make sure to set `requires_grad = True`

In [6]:
def check_valid(module_name, prefix = "transformer.h", start_layer = 1):
    if(module_name in [final_layer_norm, unembed]):
        return True
    for idx in range(start_layer, mt.num_layers):
        if(module_name.startswith(f"{prefix}.{idx}")):
            return True
    return False

need_gradients = {
    n: p
    for n, p in part_model.named_parameters()
    if check_valid(
        n, prefix = layer_name_format[:-3]
    )
}

for n, w in part_model.named_parameters():
    if(n in need_gradients):
        w.requires_grad = True
    else:   
        w.requires_grad = False

## Calculate the `RelationOperator` (Jacobian and Biases)

In [8]:
space_needle = estimate.estimate_relation_operator_neox(
    model, part_model,
    tokenizer,
    "The Space Needle",
    "{} is located in the country of",
    layer=27,

    layer_name_format = layer_name_format,
    final_layer_norm = final_layer_norm,
    unembed = unembed,
)

space_needle.misc

prompt >>  The Space Needle is located in the country of
h_token_idx >>  3
gpt_neox.layers.0  << original gpt_neox.layers.22
replacing gpt_neox.layers.5 outputs
Calculating Jacobians ...


100%|██████████| 6144/6144 [17:56<00:00,  5.71it/s]


{'Jh_norm': 107.25,
 'bias_norm': 895.0,
 'h_info': {'h_index': 3, 'token_id': 282, 'token': 'le'},
 'consider_residual': False}

In [9]:
test_cases = [
    ("The Space Needle", -1, "United States"),
    ("The Great Wall", -1, "China"),
    ("Niagara Falls", -2, "Canada"),
    ("Valdemarsvik", -1, "Sweden"),
    ("Kyoto University", -2, "Japan"),
    ("Hattfjelldal", -1, "Norway"),
    ("Ginza", -1, "Japan"),
    ("Sydney Hospital", -2, "Australia"),
    ("Mahalangur Himal", -1, "Nepal"),
    ("Higashikagawa", -1, "Japan"),
    ("Trento", -1, "Italy"),
    ("Taj Mahal", -1, "India")
]

def evaluate_against_test_cases(relation):
    for subject, subject_token_index, target in test_cases:
        objects = relation(
            subject,
            subject_token_index=subject_token_index,
            device=model.device,
            return_top_k=5,
        )
        print(f"{subject}, target: {target}   ==>   predicted: {objects}")

In [10]:
evaluate_against_test_cases(space_needle)

The Space Needle, target: United States   ==>   predicted: [' Washington', ' Seattle', ' the', ' Malaysia', ' Canada']
The Great Wall, target: China   ==>   predicted: [' Seattle', ' Washington', ' the', '\n', ' Japan']
Niagara Falls, target: Canada   ==>   predicted: [' Seattle', ' Washington', ' the', ' Japan', ' glass']
Valdemarsvik, target: Sweden   ==>   predicted: [' Seattle', ' Washington', ' the', ' Japan', ' glass']
Kyoto University, target: Japan   ==>   predicted: [' Japan', ' Seattle', ' Washington', ' the', ' Tokyo']
Hattfjelldal, target: Norway   ==>   predicted: [' Seattle', ' Washington', ' the', ' glass', ' Japan']
Ginza, target: Japan   ==>   predicted: [' Japan', ' Seattle', ' Washington', ' the', ' Tokyo']
Sydney Hospital, target: Australia   ==>   predicted: [' Seattle', ' Washington', ' the', '\n', ' origin']
Mahalangur Himal, target: Nepal   ==>   predicted: [' Seattle', ' Washington', ' the', '\n', ' glass']
Higashikagawa, target: Japan   ==>   predicted: [' Jap